<a href="https://colab.research.google.com/github/lubianat/wikidata_markers/blob/master/reconcile_cell_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd

In [42]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [43]:
worksheet = gc.open('Biocuration of Cell Classes for Wikidata' ).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)

[['label', 'subclass of', 'stated in', 'described by source', 'aliases', 'anatomical location', 'develops from', 'instance of'], ['mouse vas deferens epithelial cell', 'Q15176415', 'Q98166912', '', '', 'Q735505', '', ''], ['mouse Amylin-positive epiditymal clear cells', 'Q106634206', 'Q98166912', 'Q98166912', 'Amylin-positive clear cells', 'Q1973610', '', ''], ['mouse vas deferens muscle cell', 'Q428914', 'Q98166912', '', '', 'Q735505', '', ''], ['human AS DCs', 'human dendritic cell', 'Q34555562', 'Q34555562', 'DC5 | AXL+SIGLEC6+population', 'Q7873', '', ''], ['human cDC-like  AS DCs', 'human AS DCs', 'Q34555562', 'Q34555562', 'CD123+CD11c–/lo AS DCs', 'Q7873', '', ''], ['human pDC-like AS DCs', 'human AS DCs', 'Q34555562', 'Q34555562', 'CD123loCD11c+ AS DCs', 'Q7873', '', ''], ['human cDC progenitor', 'cell', 'Q34555562', 'Q34555562', 'human circulating cDC progenitor', 'Q7873', '', ''], ['human blasticplasmacytoid DC neoplasia ', 'cell', 'Q34555562', 'Q34555562', 'BPDCN cells', 'Q78

In [44]:
# From https://stackoverflow.com/questions/31328861/python-pandas-replacing-header-with-top-row
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header


In [45]:
instance_dict = {
    "cell type":"Q189118",
    "cell class":"Q104852483",
    "cell state":"Q104539563"
}

In [46]:
df

,label,subclass of,stated in,described by source,aliases,anatomical location,develops from,instance of
1,mouse vas deferens epithelial cell,Q15176415,Q98166912,,,Q735505,,
2,mouse Amylin-positive epiditymal clear cells,Q106634206,Q98166912,Q98166912,Amylin-positive clear cells,Q1973610,,
3,mouse vas deferens muscle cell,Q428914,Q98166912,,,Q735505,,
4,human AS DCs,human dendritic cell,Q34555562,Q34555562,DC5 | AXL+SIGLEC6+population,Q7873,,
5,human cDC-like AS DCs,human AS DCs,Q34555562,Q34555562,CD123+CD11c–/lo AS DCs,Q7873,,
...,...,...,...,...,...,...,...,...
191,human idiling acinar cells,Q101405107,Q102206138,,acinar-i,,,
192,fetal cell,cell,Q104130336,,,,,
193,ileal cell,cell,Q104130336,,,,,
194,pediatric terminal ileal cell,ileal cell,Q104130336,,,,,


In [48]:
import re

number_of_create_statements = 0
for index, row in df.iterrows():

    print("CREATE")

    label = row["label"]
    print(f'LAST|Len|"{label}"')

    stated_in = row["stated in"]
    if "human" in label:
      description = "Cell type in Homo sapiens"
      taxon = "Q15978631"
      print(f"LAST|P703|{taxon}|S248|{stated_in}")

    elif "mouse" in label:
      description = "Cell type in Mus musculus"
      taxon = "Q83310"
      print(f"LAST|P703|{taxon}|S248|{stated_in}")

    elif "zebrafish" in label:
      description = "Cell type in Danio rerio"
      taxon = "Q169444"
      print(f"LAST|P703|{taxon}|S248|{stated_in}")

    else:
      description = "Cell type"
    
    print(f'LAST|Den|"{description}"')
    
    if row["aliases"] != "":
        aliases = row["aliases"] 
        print(f'LAST|Aen|"{aliases}"')


    described_by_source = row["described by source"]

    if row["instance of"] != "":
      instance = instance_dict[row["instance of"]]
    else: 
      instance = instance_dict["cell type"]
    print(f"LAST|P31|{instance}|S248|{stated_in}")

    try:
        if  re.findall("Q[0-9]*", row["subclass of"]):
            subclass = row["subclass of"]
        else:
            subclass = subclass_dict[row["subclass of"]]            
        print(f"LAST|P279|{subclass}|S248|{stated_in}")

        try:
            part_of = part_of_dict[row["anatomical location"]]
            print(f"LAST|P927|{part_of}|S248|{stated_in}")
        except:
            pass





        if row["described by source"] != "":
            print(f'LAST|P1343|{described_by_source}')
        print("")

        number_of_create_statements += 1
        
    except:
        print("")
        print("failed: ")
        print(row)
        break
    print(" ")

CREATE
LAST|Len|"mouse vas deferens epithelial cell"
LAST|P703|Q83310|S248|Q98166912
LAST|Den|"Cell type in Mus musculus"
LAST|P31|Q189118|S248|Q98166912
LAST|P279|Q15176415|S248|Q98166912

 
CREATE
LAST|Len|"mouse Amylin-positive epiditymal clear cells"
LAST|P703|Q83310|S248|Q98166912
LAST|Den|"Cell type in Mus musculus"
LAST|Aen|"Amylin-positive clear cells"
LAST|P31|Q189118|S248|Q98166912
LAST|P279|Q106634206|S248|Q98166912
LAST|P1343|Q98166912

 
CREATE
LAST|Len|"mouse vas deferens muscle cell"
LAST|P703|Q83310|S248|Q98166912
LAST|Den|"Cell type in Mus musculus"
LAST|P31|Q189118|S248|Q98166912
LAST|P279|Q428914|S248|Q98166912

 
CREATE
LAST|Len|"human AS DCs"
LAST|P703|Q15978631|S248|Q34555562
LAST|Den|"Cell type in Homo sapiens"
LAST|Aen|"DC5 | AXL+SIGLEC6+population"
LAST|P31|Q189118|S248|Q34555562

failed: 
0
label                                  human AS DCs
subclass of                    human dendritic cell
stated in                                 Q34555562
described by sou

In [49]:
for i in range(number_of_create_statements):
  print(i)
  worksheet.delete_row(2)


0
1
2


In [ ]:
number_of_create_statements=1

{'replies': [{}],
 'spreadsheetId': '1SPgKpMEwHJSD-7M9I2hyVLJo7dlNk8maiL4SQhBGtXs'}